In [19]:
import requests
import os

NOTION_API_KEY = os.environ['NOTION_API_KEY']
NOTION_DATABASE_ID = os.environ['NOTION_DATABASE_ID']
NOTION_TEMPLATE_PAGE_ID = os.environ['NOTION_TEMPLATE_PAGE_ID']

In [20]:
from notion_client import Client

notion = Client(auth=NOTION_API_KEY)

new_page = notion.pages.create(
    parent={"database_id": NOTION_DATABASE_ID},
    properties={
        "名前": {
            "title": [
                {
                    "type": "text",
                    "text": {"content": "テンプレ複製テスト"}
                }
            ]
        },
        "Progress": {
            "status": {"name": "未読"}  # ステータス名はDBと一致する必要あり
        },
    }
)
print(new_page["id"])  # ← ページID取得

2755fed5-c66a-81be-8090-d96bcf974799


In [24]:
def create_page():
    notion = Client(auth=NOTION_API_KEY)

    new_page = notion.pages.create(
        parent={"database_id": NOTION_DATABASE_ID},
        properties={
            "名前": {"title": [{"text": {"content": "テスト"}}]},
            "Progress": {"status": {"name": "未読"}},
            "Authors": {"rich_text": [{"text": {"content": "田中さん"}}]},
            "Time": {"rich_text": [{"text": {"content": "1時"}}]},
            "URL": {"url": "https://www.notion.so/2755fed5c66a80cdbad6d9ac47387148"},
        },
    )
    return new_page["id"]

new_page_id = create_page()